# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough.assign
- itemgetter()
- session ID를 사용하여 대화(Multi Turn Conversation)를 진행하세요.
- LLM 모델은 과거 대화를 기억하지 못합니다.

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from operator import itemgetter
from dotenv import load_dotenv

## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
    ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)
## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    max_tokens=512, 
)

## 6. Trim 설정
trimmer = trim_messages(
    max_tokens=50, # 최대 토근 수 제한
    strategy="last", # 최근 메시지부터 시작해서 자르기
    token_counter=llm, # 모델 기준으로 토근 수 계산
    include_system=True, # 시스템 메시지 포함
    allow_partial=False, # 메시지 자르기 제한
    start_on="human", # Human 부터 자르기 시작
)

## 7. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)
    | prompt 
    | llm
)

## 8. 세션별 대화 이력(히스토리)을 저장할 임시 메모리 저장소
store = {}

## 9. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행(Multi Turn)
while True:
    query = input('육아휴직과 관련된 질문하세요 >> ')
    if query.upper() == 'S':
        print('종료합니다')
        break
 
    with_message_history.invoke(
        {'law': law, 'query': query},
        config={'configurable': {'session_id': 'abc'}}
    )
    
    print('\n' + '=' * 100 + '\n')

In [9]:
for message in get_history('abc').messages:
    print(f'[{message.type.upper()}]: {message.content}\n')

[HUMAN]: 육아휴직 기간은?

[AI]: 육아휴직의 기간은 1년 이내로 하며, 필요한 경우 부 또는 모가 같은 자녀를 대상으로 각각 3개월 이상 육아휴직을 사용한 경우, 한부모가족의 부 또는 모, 그리고 고용노동부령으로 정하는 장애아동의 부 또는 모는 6개월 이내에서 추가로 육아휴직을 사용할 수 있습니다. (제19조 2항)

[HUMAN]: 육아휴직 조건은?

[AI]: 육아휴직을 신청할 수 있는 조건은 다음과 같습니다:

1. **임신 중인 여성 근로자가 모성을 보호하거나**, 
2. **만 8세 이하 또는 초등학교 2학년 이하의 자녀**(입양한 자녀를 포함)를 양육하기 위하여 근로자가 신청하는 경우

이에 해당하는 경우 사업주는 육아휴직을 허용해야 합니다. 다만, 대통령령으로 정하는 경우에는 그러하지 않을 수 있습니다. (제19조 제1항)

[HUMAN]: 육아휴직 대상자는?

[AI]: 육아휴직 대상자는 임신 중인 여성 근로자이거나, 만 8세 이하 또는 초등학교 2학년 이하의 자녀를 양육하는 근로자입니다. 이 경우에는 입양한 자녀도 포함됩니다. (남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 제19조 제1항) 다만, 대통령령으로 정하는 경우 예외가 있을 수 있습니다.

[HUMAN]: 육아휴직 수당은?

[AI]: 남녀고용평등과 일·가정 양립 지원에 관한 법률에서는 육아휴직 수당에 대한 직접적인 언급은 없지만, 제20조에 따르면 국가는 사업주가 근로자에게 육아휴직이나 육아기 근로시간 단축을 허용한 경우 그 근로자의 생계비용과 사업주의 고용유지비용의 일부를 지원할 수 있다고 명시하고 있습니다. 이는 육아휴직을 사용하는 근로자가 경제적으로 지원을 받을 수 있도록 하는 조항으로 이해할 수 있습니다. (제20조). 

따라서, 실제 육아휴직 수당에 대한 상세한 사항은 관련 법령이나 사회보험제도에 의해 규정될 수 있으며, 이는 별도의 법률이나 제도를 통해 확인하셔야 할 것입니다.

[HUMAN]: 첫 번째 질문은?

[AI]: 육아휴직과 관련하여 궁금하신 점